In [1]:
%matplotlib notebook

import logging
import time
from importlib import reload

from matplotlib import pyplot as plt
from matplotlib import colors, cm
import numpy as np

import qcodes
import qcodes as qc
from qcodes.instrument.parameter import Parameter
from qcodes.dataset.measurements import Measurement
from qcodes.dataset.experiment_container import new_experiment

In [2]:
# %qtconsole

# Custom imports

In [3]:
from pytopo.awg_sequencing import broadbean as bbtools
from pytopo.awg_sequencing import awg_tools
from pytopo.rf.alazar import acquisition_tools
from pytopo.rf.alazar import awg_sequences
from pytopo.rf.alazar import acquisition_controllers

from plottr import qcodes_dataset
from plottr.qcodes_dataset import QcodesDatasetSubscriber

AttributeError: module 'numba' has no attribute 'jit'

In [ ]:
from qcodes.instrument.parameter import Parameter
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_export import get_data_by_id

from pytopo.sweep.base import Nest, Chain 
from pytopo.sweep.decorators import getter, setter
from pytopo.sweep import sweep, do_experiment, hardsweep, measure

from qcodes.math.field_vector import FieldVector
from pytopo.field_optimization.movement import FieldOptimizationProblem 
from pytopo.cQED import softwarespec as ss; reload(ss)

# Measurement functions

## Qubit measurements

In [ ]:



# xtest=np.arange(-100,100.,1)
# ytest=1.0/np.pi*(2.0/((xtest-3)**2+2.0**2))

# plt.plot(xtest,ytest,'.')
# output=fit_lorentzian(xtest,ytest)
# output.plot_fit()
# output.best_values['sigma']
# output.best_values['center']



## Get resonator trace and plot

In [ ]:
from functools import partial
from qcodes.math.field_vector import FieldVector

def get_resonator_freq(): #helper function to pass variables into resonator spec
    global xvals
    return get_resonator_spec_and_fit(xvals)

def get_resonator_FWHM(): #helper function to pass variables into resonator spec
    global xvals
    _,_,fitout=get_resonator_spec_and_fit(xvals)
    return 2*fitout.params['sigma'].value

def measure_and_plot_resonator(ax, line): #plots the trace
    liney, phasey, fitout = get_resonator_freq()
    time.sleep(0.5)
    
    line.set_ydata(liney)
    ax.relim()
    ax.autoscale_view(True, True, True)
    ax.get_figure().canvas.draw()
    
    return fitout.params['center'].value

# Init

## Configurables

In [ ]:
SAMPLE = "181002_67_d1_CD20190121"

## Initialize instruments

In [ ]:
%run -i "D:\OneDrive\BF2\Code\BF2\BF2\init_station_cqed.py"

In [ ]:
numdacs = int(station.ivvi._numdacs)
for i in range(1, numdacs+1):
    station.ivvi.parameters['dac{}'.format(i)].step = 10
    station.ivvi.parameters['dac{}'.format(i)].post_delay = 0.001

In [ ]:
with station.alazar.syncing():
    station.alazar.clock_source('EXTERNAL_CLOCK_10MHz_REF')
    station.alazar.external_sample_rate(int(500e6))
    station.alazar.channel_range1(0.4)
    station.alazar.channel_range2(0.4)
    station.alazar.sample_rate(int(5e8))
    
station.hetsrc.RF.power(-55)
station.LO.power(13.0)
station.hetsrc.IF(25e6)
#station.hetsrc.frequency(4.577e9)

#station.TWPA.power(8.5)
#station.TWPA.frequency(8.0123e9)

station.LO.frequency(), station.RF.frequency(), station.TWPA.frequency()

## Setting up Alazar acquisition controllers

In [ ]:
# controller for raw acquisition
raw_acq = instools.create_inst(acquisition_controllers.RawAcqCtl, 
                               'raw_acq', 'alazar',
                               allocate_samples=5e8, 
                               force_new_instance=True)
add2station(station, raw_acq)

# regular post-measurement IQ demod controller
post_iq_acq = instools.create_inst(acquisition_controllers.PostIQCtl, 
                                   'post_iq_acq', 'alazar',
                                   allocate_samples=5e8, 
                                   force_new_instance=True)
add2station(station, post_iq_acq)
post_iq_acq.demod_frq(25e6)

# controller that demodulates after the measurement, but does NOT integrate in time.
post_demod_acq = instools.create_inst(acquisition_controllers.PostDemodCtl, 
                                      'post_demod_acq', 'alazar',
                                      allocate_samples=5e8, 
                                      force_new_instance=True)
add2station(station, post_demod_acq)
post_demod_acq.demod_frq(25e6)

# controller that can step frequencies during acquisition.
softsweep_ctl = instools.create_inst(ss.SoftSweepCtl,
                                     'softsweep_ctl', 'alazar',
                                     allocate_samples=5e8, 
                                     force_new_instance=True)
add2station(station, softsweep_ctl)
softsweep_ctl.demod_frq(25e6)

## Essential AWG Sequences

In [ ]:
from pytopo.rf.alazar.awg_sequences import TriggerSequence

TriggerSequence.chan_map = {
    1 : ['pulse', 'ats_trigger', 'ro_trigger'],
    2 : [None, 'qb_trigger', None],
}
TriggerSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

# Setup and expt. specific tools

## Helper functions for dac assignments

In [ ]:
dac_cutter1 = station.ivvi.dac1
dac_cutter14= station.ivvi.dac2
dac_plunger3 = station.ivvi.dac7
dac_plunger7 = station.ivvi.dac4
dac_plunger16 = station.ivvi.dac5
dac_plunger20 = station.ivvi.dac6

@setter(('plungers', 'mV'),)
def set_plungers_all(val):
    """
    Set both plungers (left and right) to val (in mV).
    """
    dac_plunger20(val)
    dac_plunger3(val)
    dac_plunger16(val)
    dac_plunger20(val)
    
    return val

## Helper functions to sweep magnetic field in cartesian (for using Oxford magnet)

In [ ]:
xconvert = 68.3/station.mgnt.GRPX.ATOB()
yconvert = 67.7/station.mgnt.GRPY.ATOB()
zconvert = 11.96/station.mgnt.GRPZ.ATOB()

def get_real_field_x():
    return station.mgnt.GRPX.field()/xconvert

def get_real_field_y():
    return station.mgnt.GRPY.field()/yconvert

def get_real_field_z():
    return station.mgnt.GRPZ.field()/zconvert

@setter(('magnet_x', 'T'),) 
def set_magnet_x(val):
    """
    Set magnet x
    """
    realxval = xconvert*val
    station.mgnt.x_target(realxval)
    station.mgnt.ramp(mode='safe')
    return val

@setter(('magnet_y', 'T'),)
def set_magnet_y(val):
    """
    Set magnet y
    """
    realyval = yconvert*val
    station.mgnt.y_target(realyval)
    station.mgnt.ramp(mode='safe')
    return val

@setter(('magnet_z', 'T'),)
def set_magnet_z(val):
    """
    Set magnet z
    """
    realzval = zconvert*val
    station.mgnt.z_target(realzval)
    station.mgnt.ramp(mode='safe')
    return val

@setter(('magnet_z_aligned', 'T'),)
def set_magnet_z_aligned(val):
    """
    Set magnet z to val and magnet x to val/50
    """
    realzval = zconvert*val
    station.mgnt.z_target(realzval)
    realxval = xconvert*val/50
    station.mgnt.x_target(realxval)
    station.mgnt.ramp(mode='safe')
    return val

# EVERYTHING ABOVE HERE SHOULD BE RUN FOR INITIALIZATION

# Resonator spec

In [ ]:
station.alazar.sample_rate()

In [ ]:
station.RF.status()

In [ ]:
station.RF.on()
station.LO.on()
station.hetsrc.RF.power(-55)
station.hetsrc.IF(25e6)
ss.setup_single_averaged_IQpoint(0.5e-3, 1e-3, setup_awg=True, 
                                 post_integration_delay=0.1e-3, 
                                 allocated_buffers=10)

# station.TWPA.on() #we don't have a BNC T at the moment, we can remove this once we split the marker for the cavity

so = sweep(station.hetsrc.frequency, np.arange(3.9e9, 4.0e9, 0.1e6))(
    measure(ss.get_single_averaged_IQpoint_chanA),
)

result = do_experiment(f"resonator_spec/{SAMPLE}", so, live_plot=True)

# station.TWPA.off()

In [ ]:
station.ivvi.set_dacs_zero()

In [ ]:
station.mgnt.x(0.0)

## Standard resonator spec

In [ ]:
from pytopo.rf.alazar import acquisition_tools
    
station.alazar.clear_buffers()
acquisition_tools.simple_alazar_setup_ext_trigger(256, 1, 1)

In [ ]:
dac_cutter14()

In [ ]:
station.qubsrc.output_rf('OFF')

In [ ]:
station.fg.ch1.state('off')

In [ ]:
station.RF.off()

In [ ]:
# Standard resonator spec
station.qubsrc.off()
station = qcodes.Station.default 
station.hetsrc.RF.power(-55)
station.RF.on()

station.TWPA.off()
so = ss.measure_soft_time_avg_spec(np.arange(4.81e9, 4.83e9, 0.5e6), station.hetsrc, time_bin=0.5e-3,                                   integration_time = 30e-3,
                                   setup_awg=True, verbose=True)


result = do_experiment(f'resonator_spec/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
station.awg.ch1_m1_high(1.4)

In [ ]:
station.hetsrc.RF.power(-65)
hetfreqs = np.linspace(5.29e9, 5.32e9, 101)
#station.TWPA.on()
ctl = ss.setup_soft_sweep(hetfreqs, station.hetsrc.frequency, integration_time=30e-3)
testout = get_resonator_spec_and_fit(hetfreqs)
#station.TWPA.off()

plt.figure()
plt.plot(hetfreqs,testout[0]**2,'o')
plt.plot(hetfreqs,testout[2].best_fit)
plt.xlabel('Frequency [GHz]')
plt.ylabel('Magnitude**2 [a.u.]')
plt.title('Center = %.4f GHz, FWHM = %.2f MHz, Q = %.2f' % (testout[2].params['center'].value*1e-9, 2*testout[2].params['sigma'].value*1e-6, testout[2].params['center'].value/(2*testout[2].params['sigma'].value)))

### Plots

In [ ]:
from qcodes import load_by_id
#ds1 = load_by_id(142)
ds1 = load_by_id(541)
ds2 = load_by_id(542)  # TWPA off
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
freqs2 = ds2.get_data('frequency')
mags2 = ds2.get_data('signal_magnitude')
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))

plt.figure(6)
for i in range(1):
    plt.plot(freqs2[i][0][:],mags2[i][:], 'c', label='twpa off')
    plt.plot(freqs1[i][0][:],mags1[i][:], 'b')
    plt.grid()
    plt.legend()

## Live-monitor of resonator spec

In [ ]:
fvals = np.linspace(5.29e9, 5.33e9, 51)
ctl = ss.setup_soft_sweep(fvals, station.hetsrc.frequency, integration_time=30e-3)

def monitor_resonator_spec(ax, fvals, data_line=None):
    mag, phase, fitout = get_resonator_spec_and_fit(fvals)
    
    if data_line is None:
        data_line = ax.plot(fvals, mag**2)[0]
    else:
        data_line.set_ydata(mag**2)
    
    ax.relim()
    ax.autoscale_view(True, True, True)
    ax.get_figure().canvas.draw()
    
    return data_line
    

fig, ax = plt.subplots(1,1)

data_line = None
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')

for i in range(10):
    data_line = monitor_resonator_spec(ax, fvals, data_line=data_line)

## Resonator spec vs power

In [ ]:
station = qcodes.Station.default #resonator spec vs power
spec_frequencies=np.arange(3.94e9, 4.0e9, 0.5e6)
spec_src=station.hetsrc.frequency
spec_powers=np.arange(-70,-30,2)

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src, 
                          time_bin=0.5e-3, integration_time=30e-3)

#station.TWPA.on()
#station.TWPA.off()
so = sweep(station.hetsrc.RF.power,spec_powers)(
    ss.measure_soft_time_avg_spec(spec_frequencies, station.hetsrc, setup_awg=False, time_bin=0.5e-3,
                                   integration_time = 30e-3))

result = do_experiment(f'resonator_specvspower/{SAMPLE}', so, live_plot=True)


In [ ]:
from matplotlib.colors import LogNorm
data = qc.dataset.data_export.load_by_id(151)
freqs = data.get_data('frequency')
powers = data.get_data('RF_power')
signal = data.get_data('signal_magnitude')

x = freqs[0][0]
y = np.array(powers)[:,0].T
X,Y = np.meshgrid(x,y)
Z = np.array(signal)[:,0,:]

Znorm = Z
for i in range(len(X[:,0])):
    Znorm[i,:] = Z[i,:]/np.mean(Z[i,-15:-1])

plt.figure(figsize=(3, 3))
plt.pcolor(X,Y,Z,cmap='viridis', norm=LogNorm(vmin=Z.min(), vmax=Z.max()))
plt.xlabel('Frequency (GHz)')
plt.ylabel('Power (dBm)')
#plt.axis([x.min(),x.max(),y.min(), y.max()])



# TWPA tuning

In [ ]:
station.TWPA.power(5.5)
station.TWPA.frequency(8.012e9)

In [ ]:
# station.hetsrc.RF.power(-60)
# so = sweep(station.TWPA.power, np.arange(9, 11, 0.1))(
#     softspec(np.linspace(5.3e9, 5.326e9, 27), station.hetsrc, navgs=10, setup_awg=False)
# )

# result = do_experiment(f'TWPA_resonator_spec/{SAMPLE}', so, live_plot=True)

In [ ]:
dac_cutter14(698)

In [ ]:
def get_snr_from_max(trace, background_range=(0, 20)):
    idx0, idx1 = background_range
    S = np.abs(np.mean(trace[idx0:idx1])-trace.min())
    N = np.abs(np.std(trace[idx0:idx1]))
    return S, N

@getter(('SNR', ''))
def SNR():
    mag, phase = ss.get_soft_sweep_trace()
    trace = mag * np.exp(-1j*phase)
    s, n = get_snr_from_max(trace)
    return s/n


station.qubsrc.output_rf('OFF')
spec_frequencies = np.arange(4.4e9, 4.6e9, 1e6)
spec_src = station.hetsrc
integration_time = 20e-3

#pump_amps = np.linspace(4.5, 6.0, 16)
pump_frqs = np.arange(7.95e9, 7.98e9, 1e6)

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3,
                         verbose=True, allocated_buffers=100)

station.TWPA.on()
so = sweep(station.TWPA.frequency, pump_frqs)(
         ss.measure_soft_time_avg_spec(spec_frequencies, spec_src,time_bin=0.5e-3, integration_time=30e-3,
                                       setup_awg=False),
        #measure(SNR),
    )

result = do_experiment(f'tune_TWPA_resonator_spec/{SAMPLE}', so, live_plot=True)
station.TWPA.off() #the TWPA power seems to warm up the fridge so let's turn it off when we are not measuring

In [ ]:
station.RF.power()

# Resonator vs Gate Response

## Measure at a single frequency

### Vs cutter

In [ ]:
#setup_single_averaged_IQpoint(0.1e-3, 10e-3, setup_awg=True)

cur_cutter14 = dac_cutter14()
#station.hetsrc.frequency(5.31347e9)
#station.TWPA.on() #we don't have a BNC T at the moment, we can remove this once we split the marker for the cavity
so = sweep(dac_cutter14, np.arange(cur_cutter14, 1800., 1.))(
    measure(ss.get_single_averaged_IQpoint_chanA),
)

result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
station.hetsrc.frequency(5.310e9)
station.hetsrc.RF.power(-55)

cur_cutter = dac_cutter1()

so = ss.measure_soft_gate_sweep(
    np.arange(cur_cutter, cur_cutter+5, 0.1), dac_cutter,
    integration_time=10e-3, setup=True,
)

station.TWPA.on()
result = do_experiment(f"soft_sweep_gate/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

### Vs plungers

In [ ]:
station.hetsrc.frequency(5.305e9)
cur_plunger = dac_leftplunger()

ss.setup_single_averaged_IQpoint(0.1e-3, 30e-3, setup_awg=True)
so = sweep(set_plungers, np.arange(cur_plunger,-400,1))(
    measure(ss.get_single_averaged_IQpoint_chanA),
)

#station.TWPA.on()
result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
#station.TWPA.off()

In [ ]:
# station.hetsrc.frequency(5.305e9)
# station.hetsrc.RF.power(-70)

# cur_plunger = dac_leftplunger()

# so = ss.measure_soft_gate_sweep(
#     np.arange(cur_plunger, cur_plunger+5, 0.1), dac_cutter,
#     integration_time=10e-3, setup=True,
# )

# station.TWPA.on()
# result = do_experiment(f"soft_sweep_gate/{SAMPLE}", so, live_plot=True)
# station.TWPA.off()

### Fixed frequency scans (view each point)

In [ ]:
station.TWPA.frequency()

In [ ]:
%%time
#Fixed frequency scan if you want to see each point individually

station.hetsrc.frequency(5.31347e9+4e6)
cur_plunger = dac_leftplunger()
cur_cutter = dac_cutter()
cutter_vals = np.arange(300,1500.,5)
plunger_vals = np.arange(-300,100.,10)


setup_single_averaged_IQpoint(0.1e-3, 30e-3, setup_awg=True)
so = sweep(dac_cutter, cutter_vals)(sweep(set_plungers, plunger_vals)(
    measure(get_single_averaged_IQpoint_chanA),
))

station.TWPA.on()
result = do_experiment(f"fixed_frequency_slow/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

### Fixed frequency scan (only see 1d, faster)

In [ ]:
%%time
#Fast Fixed frequency scan if not seeing each 1d line is ok with you

station.hetsrc.frequency(4.816e9)
cur_plunger = dac_plunger16()
cur_cutter = dac_cutter1()
cutter_vals = np.arange(300,600.0,2)[::-1]
plunger_vals = np.arange(-1000,1000.,5)


ctl = ss.setup_soft_sweep(plunger_vals, set_plungers_all,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

so = sweep(dac_cutter1, cutter_vals)(ss.measure_soft_gate_sweep(
    plunger_vals, set_plungers_all,time_bin=0.5e-3,post_integration_delay = 100e-6,
    integration_time=30e-3, setup_awg=False))

#station.TWPA.on()
result = do_experiment(f"fixed_frequency_fast/{SAMPLE}", so, live_plot=True)
#station.TWPA.off()

In [ ]:
station.TWPA.off()

## Take full resonator scan vs gate

### Vs cutter

In [ ]:
set_plungers_all(-1000)

In [ ]:
cur_cutter1 = dac_cutter1()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.8e9, 4.83e9, 1e6)
spec_src=station.hetsrc

station.qubsrc.off()

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()

so = sweep(dac_cutter1, np.arange(cur_cutter1, 493, -1.))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_cutter_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

### Vs plungers

In [ ]:
cur_plunger = dac_plunger16()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.80e9, 4.84e9, 0.2e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=10e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()

so = sweep(set_plungers_all, np.arange(cur_plunger,-900.0,-4))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=10e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)
result = do_experiment(f"resonator_plunger_response/{SAMPLE}", so, live_plot=True)

station.TWPA.off()

In [ ]:
#Use this code to take slices of full resonator vs plunger at multiple different cutter voltages

cur_plunger = dac_plunger16()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.80e9, 4.84e9, 0.2e6)
cur_cutter = dac_cutter1()
cutter_voltages = np.arange(cur_cutter,600.0,10)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=10e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
for i in np.arange(0,28,2):
    dac_cutter1(cutter_voltages[i])
    so = sweep(set_plungers_all, np.arange(-1000.0,500.0,5))(
        ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=10e-3,
                                      setup_awg=False, verbose=False, allocated_buffers=100,
                                      post_integration_delay=100e-6)
    )
    result = do_experiment(f"resonator_plunger_response/{SAMPLE}", so, live_plot=True)
    dac_cutter1(cutter_voltages[i+1])
    so = sweep(set_plungers_all, np.arange(500.0,-1000.0,-5))(
        ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=10e-3,
                                      setup_awg=False, verbose=False, allocated_buffers=100,
                                      post_integration_delay=100e-6)
    )
    result = do_experiment(f"resonator_plunger_response/{SAMPLE}", so, live_plot=True)

station.TWPA.off()

In [ ]:
set_plungers_all(-500)

In [ ]:
cur_plunger = dac_plunger16()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(5.23e9, 5.27e9, 1.0e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(dac_plunger7, np.arange(cur_plunger,1000,20))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
dac_plunger7(-1000)

In [ ]:
cur_plunger = dac_plunger16()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(5.23e9, 5.27e9, 1.0e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(dac_plunger16, np.arange(cur_plunger,2000,20))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()
dac_plunger16(-1000)

In [ ]:
cur_plunger = dac_plunger16()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(5.23e9, 5.27e9, 1.0e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(dac_plunger20, np.arange(cur_plunger,1000,20))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()
dac_plunger20(-1000)

In [ ]:
station.ivvi.set_dacs_zero()

### Plots

In [ ]:
from qcodes import load_by_id
ds1 = load_by_id(485)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
ds2 = load_by_id(382)
freqs2 = ds2.get_data('frequency')
mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))

plt.figure(6)
for i in range(1):
    plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
    plt.grid()
#plt.xlim(3e9,8e9)


#plt.figure(7)
#plt.imshow(np.transpose(mags1)[-1:0:-1],aspect='auto',extent = [-500, 1800., freqs1[0][0][0],freqs1[0][0][-1]])

# Resonator vs Flux (with Yoko)

In [ ]:
#station.yoko.off()
station.yoko.source_mode('CURR')
station.yoko.current_range(0.1)
#station.yoko.current(0.00)
station.yoko.on()

In [ ]:
dac_cutter14(167)

In [ ]:
station.yoko.current()

In [ ]:

station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.795e9, 4.86e9, 0.1e6)
spec_src=station.hetsrc
#14.64 mT/Amp is the field to current ratio
applied_currents = np.arange(station.yoko.current(),0.1,0.005/14.64)
station.RF.on()
station.LO.on()

station.qubsrc.frequency(3.406e9)
station.qubsrc.on()

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=15e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)


so = sweep(station.yoko.current, applied_currents)(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=15e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_flux_response/{SAMPLE}", so, live_plot=True)
station.qubsrc.on()

# Qubit spec

## Spec vs power

In [ ]:
station = qcodes.Station.default ##spec vs power
station.hetsrc.RF.power(-55)
station.hetsrc.frequency(5.226e9)
station.qubsrc.power(-35)
qubit_frequencies = np.arange(7.0e9, 7.5e9, 5e6)
spec_src=station.qubsrc
station.fg.ch1.state('OFF')

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3)

spec_src.on()
so = ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, time_bin=0.5e-3, 
                                   integration_time=30e-3, setup_awg=False, allocated_buffers=100,
                                   post_integration_delay=100e-6)

result = do_experiment(f'qubit_spec/{SAMPLE}', so, live_plot=True)
spec_src.off()


In [ ]:
station.fg.ch1.state('OFF')

In [ ]:
station = qcodes.Station.default ##qubit spec vs power
station.hetsrc.RF.power(-60)
station.hetsrc.frequency(5.225e9)
qubit_frequencies = np.arange(7.0e9, 7.5e9, 1e6)
qubit_powers = np.linspace(-50,-50,1)
spec_src=station.qubsrc


ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.on()
so = sweep(station.qubsrc.power,qubit_powers)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
station.TWPA.off()

In [ ]:
station = qcodes.Station.default ##spec vs power
station.hetsrc.RF.power(-55)
station.hetsrc.frequency(3.967e9)
qubit_frequencies = np.arange(4.5e9, 8.5e9, 5e6)
qubit_powers = np.linspace(-25,-15,6)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(station.qubsrc.power,qubit_powers)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)

In [ ]:
station = qcodes.Station.default ##spec vs power
station.hetsrc.RF.power(-55)
station.hetsrc.frequency(3.967e9)
qubit_frequencies = np.arange(8.5e9, 10.5e9, 5e6)
qubit_powers = np.linspace(-25,-15,6)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(station.qubsrc.power,qubit_powers)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)

In [ ]:
station = qcodes.Station.default ##spec vs RESONATOR power
RF_powers = np.arange(-70,-40,2)
station.hetsrc.frequency(4.542e9)
qubit_frequencies = np.arange(6.5e9, 9e9, 10e6)
station.qubsrc.power(-20)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(station.hetsrc.RF.power,RF_powers)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_Respower/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
for f in np.arange(0., 6.0, 0.25):
    station = qcodes.Station.default ##spec vs power
    station.hetsrc.RF.power(-65)
    station.hetsrc.frequency(4.526e9)
    station.qubsrc.power(-15)
    qubit_frequencies = np.arange((4+f)*1e9, (4+f+0.25)*1e9, 1e6)
    qubit_powers = np.arange(-45,-10,4)
    spec_src=station.qubsrc

    ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3)

    #station.TWPA.on()
    spec_src.on()
    so = sweep(station.qubsrc.power,qubit_powers)(
        ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3, setup=False))

    result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
    #station.TWPA.off()

### Plot

In [ ]:
from qcodes import load_by_id
i=0
for f in np.arange(0., 6.0, 0.25):
    ds1 = load_by_id(458+i)
    #freqs1 = ds1.get_data('frequency')
    mags1 = ds1.get_data('signal_magnitude')
    #freqs2 = ds2.get_data('frequency')
    #mags2 = ds2.get_data('magnitude')
    mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
    if i== 0:
        mags = mags1
    else:
        mags = np.append(mags, mags1, axis=1)
    i += 1
    
plt.figure(8)
plt.imshow((mags)[-1:0:-1],aspect='auto',extent = [4.0, 10.0,-65,-15], vmax = 0.00014)
plt.colorbar()
plt.xlabel('qubit drive frequency (GHz)')
plt.ylabel('qubit drive power (dBm)')

In [ ]:
from qcodes import load_by_id

ds1 = load_by_id(852)
#freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
    
plt.figure()
plt.imshow((mags1)[-1:0:-1],aspect='auto',extent = [5.0, 9.5,-20,-10], vmax = 0.0045, vmin = 0.001)
plt.colorbar()
plt.xlabel('qubit drive frequency (GHz)')
plt.ylabel('qubit drive power (dBm)')

## Spec vs gates

### Coarse scans (requiring resonator frequency change)

In [ ]:
from pytopo.rf.alazar import acquisition_tools
    
station.alazar.clear_buffers()
acquisition_tools.simple_alazar_setup_ext_trigger(256, 1, 1)

In [ ]:
dac_cutter1()

In [ ]:
# COARSE SCAN VS CUTTER - Use this if you want to look at Ej vs gate
station = qcodes.Station.default
station.hetsrc.RF.power(-55)
het_freqs = np.arange(5.195e9, 5.235e9, 1e6)
station.qubsrc.power(-30)
qubit_frequencies = np.arange(6.5e9, 8.0e9, 5e6)
cur_cutter1 = dac_cutter1()
cutter1_voltages = np.arange(cur_cutter1, 205, 2)

station.qubsrc.on()

#station.TWPA.on()

ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3)

so = sweep(dac_cutter1, cutter1_voltages)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                             qubit_frequencies, station.qubsrc, 
                                             time_bin=0.5e-3, integration_time=30e-3, 
                                             hanger=True, verbose=False, allocated_buffers=100,
                                             post_integration_delay=100e-6)
) 

#note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_spec/{SAMPLE}', so, live_plot=True)

station.TWPA.off()

In [ ]:
station = qcodes.Station.default
station.hetsrc.RF.power(-55)
het_freqs = np.arange(4.53e9,4.58e9,1e6)
station.qubsrc.power(-15)
qubit_frequencies = np.arange(5.5e9, 7.5e9, 5e6)
cur_cutter14 = dac_cutter14()
cutter14_voltages = np.arange(cur_cutter14, 0.0, -2.0)

station.qubsrc.output_rf('ON')

#station.TWPA.on()

ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3)

so = sweep(dac_cutter14, cutter14_voltages)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                             qubit_frequencies, station.qubsrc, 
                                             time_bin=0.5e-3, integration_time=30e-3, 
                                             hanger=True, verbose=False, allocated_buffers=100,
                                             post_integration_delay=100e-6)
) 

#note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_spec/{SAMPLE}', so, live_plot=True)

station.TWPA.off()

In [ ]:
station.TWPA.off()

In [ ]:
station = qcodes.Station.default
station.hetsrc.RF.power(-65)
station.qubsrc.power(-15)
het_freqs = np.arange(4.520e9,4.535e9,0.5e6)
qubit_frequencies = np.arange(8.5e9, 8.8e9, 1e6)
cur_plunger = dac_leftplunger()
plunger_voltages = np.arange(cur_plunger, cur_plunger+160, 0.2)

ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, integration_time=30e-3)
so = sweep(set_plungers, plunger_voltages)(
    measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                           qub_freqs, station.qubsrc, integration_time=30e-3)) 

#note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_spec/{SAMPLE}', so, live_plot=True)

#### Plots

In [ ]:
from qcodes import load_by_id
ds1 = load_by_id(519)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
#ds2 = load_by_id(382)
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
#mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))

#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure()
plt.imshow(np.transpose(mags1)[-1:0:-1],aspect='auto',extent = [687, 715, 7.9, 9.2], vmax = 0.019, vmin = 0.010)
plt.colorbar()
plt.xlabel('cutter14 (mV)')
plt.ylabel('qubit drive frequency (GHz)')

### Fine scans (resonator does not change)

In [ ]:
station = qcodes.Station.default #charge dispersions
station.hetsrc.RF.power(-50)
station.hetsrc.frequency(4.542e9)
station.qubsrc.power(-20)
qubit_frequencies = np.arange(5.5e9, 11e9, 15e6)
cur_cutter14 = dac_cutter14()
cutter14_voltages = np.arange(500, 521, +1.0/5)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(dac_cutter14, cutter14_voltages)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, 
                                  time_bin=0.5e-3, integration_time=30e-3, 
                                  verbose=True, allocated_buffers=100,
                                  post_integration_delay=50e-6))

result = do_experiment(f'two-tone-vs-cutter/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
## Fine cutter sweep without updating the resonator position.

for f in np.arange(0., 4.5, 0.25):
    station = qcodes.Station.default
    station.hetsrc.RF.power(-65)
    station.hetsrc.frequency(4.526e9)
    station.qubsrc.power(-15)
    qubit_frequencies = np.arange((5.5+f)*1e9, (5.5+f+0.25)*1e9, 1e6)
    cutter14_voltages = np.arange(698,700,0.5)
    spec_src=station.qubsrc

    ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3)

    #station.TWPA.on()
    spec_src.on()
    so = sweep(dac_cutter14,cutter14_voltages)(
        ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3, setup_awg=False))

    result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
    #station.TWPA.off()

In [ ]:
# Fine cutter14 dependence without updating resonator

station = qcodes.Station.default
station.hetsrc.RF.power(-55)
station.hetsrc.frequency(4.55e9)
station.qubsrc.power(-25)
qubit_frequencies = np.arange(6.5e9, 8e9, 15e6)
cur_cutter14 = dac_cutter14()
cutter14_voltages = np.arange(cur_cutter14, 1230, -2)
spec_src=station.qubsrc


ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(dac_cutter14,cutter14_voltages)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, time_bin=0.5e-3,  integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=50e-6))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
station = qcodes.Station.default #charge dispersions with phase information
station.hetsrc.RF.power(-60)
station.hetsrc.frequency(5.2992e9)
station.qubsrc.power(-15)
qubit_frequencies = np.arange(6.0e9, 7.0e9, 5e6)
cur_plunger = dac_leftplunger_fine()
plunger_voltages = np.arange(0., 60., 2)
spec_src=station.qubsrc

ctl = ss.setup_single_averaged_IQpoint(0.2e-3, 30e-3, setup_awg=True)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(dac_leftplunger_fine, plunger_voltages)(sweep(station.qubsrc.frequency, qubit_frequencies)(
    measure(ss.get_single_averaged_IQpoint_chanA),
))
    
result = do_experiment(f'charge_dispersions/{SAMPLE}', so, live_plot=True)

In [ ]:
res_powers = np.arange(-70,-45,5)
spec_powers = np.arange(-30,-15,5)

for res_power in res_powers:
    for spec_power in spec_powers:
        station = qcodes.Station.default ##spec vs cutter
        station.hetsrc.RF.power(res_power)
        station.hetsrc.frequency(5.30e9)
        station.qubsrc.power(spec_power)
        qubit_frequencies = np.arange(6.0e9, 7.2e9, 10e6)
        cur_plunger = dac_leftplunger_fine()
        plunger_voltages = np.arange(cur_plunger, cur_plunger+60, 2)
        spec_src=station.qubsrc

        ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

        station.TWPA.off()
        spec_src.output_rf('ON')
        so = sweep(dac_leftplunger_fine, plunger_voltages)(
            ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, setup=False))

        result = do_experiment(f'charge_dispersions/{SAMPLE}', so, live_plot=True)
        #station.TWPA.off()

#### Plot


In [ ]:
from qcodes import load_by_id
i=0
for f in np.arange(0., 4.5, 0.25):
    ds1 = load_by_id(486+i)
    #freqs1 = ds1.get_data('frequency')
    mags1 = ds1.get_data('signal_magnitude')
    #freqs2 = ds2.get_data('frequency')
    #mags2 = ds2.get_data('magnitude')
    mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
    if i== 0:
        mags = mags1
    else:
        mags = np.append(mags, mags1, axis=1)
    i += 1
    
plt.figure(9)
plt.imshow((mags)[-1:0:-1],aspect='auto',extent = [4, 8.7,730,735], vmax = 0.007)
plt.colorbar()
plt.xlabel('qubit drive frequency (GHz)')
plt.ylabel('cutter14 (mV)')

In [ ]:
from qcodes import load_by_id

ds1 = load_by_id(828)
#freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))

plt.figure()
plt.imshow((mags1)[-1:0:-1],aspect='auto',extent = [4, 8.7, 730,732], vmax = 0.007, vmin = 0.002)
plt.colorbar()
plt.xlabel('qubit drive frequency (GHz)')
plt.ylabel('cutter14 (mV)')

## Spec vs flux

In [ ]:
station.yoko.off()
station.yoko.source_mode('CURR')
station.yoko.current_range(0.1)
station.yoko.current(0.00)
station.yoko.on()

### Coarse scan (updating heterodyne frequency)

In [ ]:
station.yoko.current()

In [ ]:
station.TWPA.off()

In [ ]:
dac_cutter14()

In [ ]:
np.concatenate([np.arange(0.3e9,2.4e9,7e6),np.arange(3.1e9,4.2e9,7e6)])

In [ ]:
station.yoko.current()

In [ ]:
station = qcodes.Station.default #qubit spec vs flux updating resonator (should basically )
station.hetsrc.RF.power(-55)
station.qubsrc.power(-25)
het_freqs = np.arange(4.805e9, 4.88e9, 1e6)
qubit_frequencies = np.arange(3.0e9,4.5e9,5e6)

#14.64 mT/Amp is the field to current ratio
fluxline_currentsDown = np.arange(station.yoko.current(),0.02,-0.002/14.64)

station.qubsrc.on()

#station.TWPA.on()
ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=15e-3)


so = sweep(station.yoko.current, fluxline_currentsDown)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                            qubit_frequencies, station.qubsrc, 
                                            time_bin=0.5e-3, integration_time=15e-3, 
                                            hanger = True, verbose=False, allocated_buffers=100,
                                             post_integration_delay=50e-6)
)

    #note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_specVsFlux/{SAMPLE}', so, live_plot=True)

In [ ]:
#this is for sweeping multiple frequency ranges
station = qcodes.Station.default
station.hetsrc.RF.power(-55)
station.qubsrc.power(-30)
het_freqs = np.arange(5.195e9, 5.26e9, 1e6)
qubit_frequencies = np.array([np.arange(0.5e9,2.5e9,5e6),np.arange(2.5e9,5.0e9,5e6),np.arange(5.75e9,8.25e9,5e6),
                              np.arange(8.25e9,10.75e9,5e6)])
fluxline_currentsUp = np.arange(0.0,0.04,0.002/14.64) #14.64 mT/Amp is the field to current ratio
fluxline_currentsDown = np.arange(0.04,0.0,-0.002/14.64)

station.qubsrc.on()

#station.TWPA.on()
ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3)

for i in np.arange(0,3,2):
    so = sweep(station.yoko.current, fluxline_currentsDown)(
        ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                                 qubit_frequencies[i], station.qubsrc, 
                                                 time_bin=0.5e-3, integration_time=30e-3, 
                                                 hanger = True, verbose=False, allocated_buffers=100,
                                                 post_integration_delay=50e-6)
    )

    #note that the two integration times must match!!

    result = do_experiment(f'optimizedqubit_specVsFlux/{SAMPLE}', so, live_plot=True)
    
    so = sweep(station.yoko.current, fluxline_currentsUp)(
        ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                                 qubit_frequencies[i+1], station.qubsrc, 
                                                 time_bin=0.5e-3, integration_time=30e-3, 
                                                 hanger = True, verbose=False, allocated_buffers=100,
                                                 post_integration_delay=50e-6)
    )

    #note that the two integration times must match!!

    result = do_experiment(f'optimizedqubit_specVsFlux/{SAMPLE}', so, live_plot=True)
station.qubsrc.off()
    
station.TWPA.off()

In [ ]:
station.TWPA.off()
station.yoko.ramp_current(-0.000, 0.001, 0.1)

#### Plots

In [ ]:
from qcodes import load_by_id
ds1 = load_by_id(828)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
#ds2 = load_by_id(382)
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
#mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))

#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure()
plt.imshow(np.transpose(mags1)[-1:0:-1],aspect='auto',extent = [-4.2e-3, 4.2e-3, 8.1, 8.55], vmax = 0.0046, vmin = 0.0040)
plt.colorbar()
plt.xlabel('yoko current (A)')
plt.ylabel('qubit drive frequency (GHz)')

## Spec vs readout frequency

In [ ]:
station = qcodes.Station.default 
station.hetsrc.RF.power(-55)
station.qubsrc.power(-25)
qubit_frequencies = np.arange(6.5e9, 9.0e9, 5e6)
readout_freq = np.arange(4.54e9, 4.59e9, +0.5e6)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)


station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(station.hetsrc.frequency, readout_freq)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=100e-6))

result = do_experiment(f'spec_vs_readout_freq/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

## Saturation spectroscopy

In [ ]:
import numpy as np
import qcodes as qc
import broadbean as bb
from broadbean.plotting import plotter

from pytopo.awg_sequencing import broadbean as bbtools
from pytopo.awg_sequencing.broadbean import BluePrints, BroadBeanSequence

ramp = bb.PulseAtoms.ramp
gaussian = bb.PulseAtoms.gaussian

class SatSpecSequence(BroadBeanSequence):
    """
    A sequence that consists of a square qubit pulse followed by a readout pulse.

    required channels:
        'ats_trigger' : marker for the alazar
        'ro_pulse' : readout marker 
        'qb_pulse' : qubit marker
    """
    name = 'SatSpec_sequence'
    
    def sequence(self, pulse_time, readout_time, cycle_time = 10e-6, cav_risetime = 0.1e-6, overlap_time = 0.0,
                 pre_pulse_time=1e-6, ncycles=2,
                 alazar_trigger_time=0.1e-6):
        """
        Parameters:
        'pulse_time' : length of the qubit pulse
        'readout_time' : length of the pulse on the readout channel
        'cycle_time' : time of each elem
        'cav_risetime' : give the cavity some time to ring up before taking data
        'overlap_time': amount of time the cavity and the qubit pulse overlap (max is the pulse_time)
        'pre_pulse_time' : some delay before the any pulses are played
        'ncycles': the number of cycles in the sequence. Typically for saturation spectroscopy we need to take
        small time bins and average many times. The AWG can't handle it if your sequence has too many elemnts
        so this is taken care of here by having 1 starting element where it waits for a trigger and then repeating
        the element the number of averages you want to take -1.
        """
        
        elements = []
        
            
        bps = bbtools.BluePrints(chan_map=self.chan_map, length=cycle_time, sample_rate=self.SR)
        bps['pulse'].insertSegment(0, ramp, (0, 0), dur=cycle_time)
        bps['qb_pulse'] = [(pre_pulse_time, pulse_time)]
        bps['ats_trigger'] = [(pre_pulse_time+pulse_time - overlap_time+cav_risetime, alazar_trigger_time)]
        bps['ro_pulse'] = [(pre_pulse_time+pulse_time - overlap_time, readout_time+2*cav_risetime)]

        elements.append(bbtools.blueprints2element(bps))
            
        
        
        bps = bbtools.BluePrints(chan_map=self.chan_map, length=cycle_time, sample_rate=self.SR)
        bps['pulse'].insertSegment(0, ramp, (0, 0), dur=cycle_time)
        bps['qb_pulse'] = [(pre_pulse_time, pulse_time)]
        bps['ats_trigger'] = [(pre_pulse_time+pulse_time - overlap_time+cav_risetime, alazar_trigger_time)]
        bps['ro_pulse'] = [(pre_pulse_time+pulse_time - overlap_time, readout_time+2*cav_risetime)]
        elements.append(bbtools.blueprints2element(bps))
        
        seq = bbtools.elements2sequence(elements, self.name)
        seq.setSequencingNumberOfRepetitions(2, ncycles-1)
        
        return seq
    
    
SatSpecSequence.chan_map = {
    1 : ['pulse', 'ats_trigger', 'ro_pulse'],
    2 : [None, 'qb_pulse', None],   
}

SatSpecSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

## THIS IS A HACK. NEVER CODE LIKE THIS. WILL INCORPORATE IT INTO SOFTWARE SPEC CORRECTLY

In [ ]:
def setup_soft_sweep_SS(values, param, time_bin=0.2e-3, integration_time=10e-3,
                     post_integration_delay=10e-6, setup_awg=True, ctl=None,
                     allocated_buffers=None, verbose=False):

    awg = qcodes.Station.default.awg
    if ctl is None:
        ctl = qcodes.Station.default.softsweep_ctl
    alazar = qcodes.Station.default.alazar

    navgs = int(integration_time / time_bin)

    if setup_awg:
        SatSpec_seq = SatSpecSequence(station.awg, SR=1e8)
        SatSpec_seq.wait = 'first'
        SatSpec_seq.setup_awg(pulse_time=5e-6, readout_time=5e-6, cav_risetime = 0.2e-6,
                              cycle_time = 12e-6, overlap_time = 5e-6,
                              ncycles = navgs)
    ctl.param = param
    ctl.values = values
    ctl.buffers_per_block(navgs)
    ctl.average_buffers(True)

    nalloc = max(2, len(values)-1)
    if allocated_buffers is not None:
        nalloc = allocated_buffers
    ctl.setup_acquisition(samples=int((time_bin-post_integration_delay) * alazar.sample_rate() // 128 * 128),
                          records=1,
                          buffers=len(values)*navgs, verbose=verbose,
                          allocated_buffers=nalloc)
    return ctl


int_time = 15e-3
t_bin = 2e-6


station.hetsrc.RF.power(-52)
station.hetsrc.frequency(5.226e9)
qubit_frequencies = np.arange(7.1e9, 7.4e9, 5e6)
spec_src=station.qubsrc
spec_src.power(-25)


ctl = setup_soft_sweep_SS(qubit_frequencies, spec_src.frequency, time_bin=t_bin, integration_time=int_time, 
                          post_integration_delay = 0.0e-6, setup_awg = True)
#alazar takes samples of int(time_bin - post integration delay )

spec_src.on()
so = ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, time_bin=t_bin, 
                                   integration_time=int_time, setup_awg=False, allocated_buffers=100,
                                   post_integration_delay=0.0e-6)

result = do_experiment(f'qubit_spec/{SAMPLE}', so, live_plot=True)
spec_src.off()

# Using magnetic field

In [ ]:
station.mgnt.GRPX.field_ramp_rate(1e-3)
station.mgnt.GRPY.field_ramp_rate(1e-3)
station.mgnt.GRPZ.field_ramp_rate(1e-3)

In [ ]:
set_magnet_x(5.7e-3)
# set_magnet_y(0.0)
# set_magnet_z(0.0)

## Resonator spec vs field

In [ ]:
cur_field_x = station.mgnt.x()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.56e9, 4.61e9, 1e6)
spec_src=station.hetsrc

station.qubsrc.output_rf('OFF')

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(station.mgnt.x, np.arange(cur_field_x, 0.0, -0.005e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.53e9, 4.55e9, 0.5e6)
spec_src=station.hetsrc

station.qubsrc.output_rf('OFF')

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(set_magnet_x, np.arange(5.9e-3, 5.08e-3, -0.002e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_gate_response/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
get_real_field_x()

## Fine qubit spec vs field (without updating resonator)

In [ ]:
dac_cutter14()

In [ ]:
get_real_field_x()

In [ ]:
set_magnet_x(0.0e-3)

In [ ]:
# FINE SCAN VS FLUX USING MAGNETIC FIELD
station = qcodes.Station.default ##spec vs field
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-60)
station.qubsrc.power(-18)
station.hetsrc.frequency(4.546e9)
qubit_frequencies = np.arange(8.5e9, 11.5e9, 10e6)
B_x = np.arange(0.0e-3, +0.8e-3, +0.005e-3)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(set_magnet_x,B_x)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
# FINE SCAN VS FLUX USING MAGNETIC FIELD
station = qcodes.Station.default ##spec vs field
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-60)
station.qubsrc.power(-18)
station.hetsrc.frequency(4.546e9)
qubit_frequencies = np.arange(5.5e9, 8.5e9, 10e6)
B_x = np.arange(0.8e-3, +0.0e-3, -0.005e-3)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(set_magnet_x,B_x)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
# FINE SCAN VS FLUX USING MAGNETIC FIELD
station = qcodes.Station.default ##spec vs field
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-60)
station.qubsrc.power(-18)
station.hetsrc.frequency(4.546e9)
qubit_frequencies = np.arange(4.5e9, 1e9, -10e6)
B_x = np.arange(0.0e-3, +0.8e-3, +0.005e-3)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(set_magnet_x,B_x)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
dac_cutter14()

In [ ]:
get_real_field_x()

In [ ]:
# FINE SCAN VS FLUX USING MAGNETIC FIELD
station = qcodes.Station.default ##spec vs field
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-60)
station.qubsrc.power(-13)
station.hetsrc.frequency(4.546e9)
qubit_frequencies = np.arange(5.3e9, 8e9, 10e6)
B_x = np.arange(0.05e-3, 0.5e-3, +0.003e-3)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(set_magnet_x,B_x)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

In [ ]:
dac_cutter14()

In [ ]:
get_real_field_x()

In [ ]:
set_magnet_x(0.2e-3)

In [ ]:
# FINE SCAN VS FLUX USING MAGNETIC FIELD
station = qcodes.Station.default ##spec vs field
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-60)
station.qubsrc.power(-13)
station.hetsrc.frequency(4.546e9)
qubit_frequencies = np.arange(2.5e9, 4.4e9, 10e6)
B_x = np.arange(0.175e-3, 0.3e-3, 0.003e-3)
spec_src=station.qubsrc

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.off()
spec_src.output_rf('ON')
so = sweep(set_magnet_x,B_x)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, integration_time=30e-3))

result = do_experiment(f'qubit_spec_vs_power/{SAMPLE}', so, live_plot=True)
#station.TWPA.off()

## Coarse qubit spec vs field (updating resonator)

In [ ]:
dac_cutter14()

In [ ]:
get_real_field_x()

In [ ]:
#set_magnet_x(0.85e-3)

In [ ]:
# COARSE SCAN VS FLUX USING MAGNETIC FIELD
cur_field_x = get_real_field_x()
station = qcodes.Station.default
station.hetsrc.RF.power(-55)
station.qubsrc.power(-30)
het_freqs = np.arange(4.537e9, 4.555e9, 1e6)
qubit_frequencies = np.arange(1e9, 10.5e9, 15e6)
B_x = np.arange(6.2e-3, 5.3e-3, -0.004e-3)

station.qubsrc.output_rf('ON')

station.TWPA.off()
ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3)

so = sweep(set_magnet_x, B_x)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                             qubit_frequencies, station.qubsrc, 
                                             time_bin=0.5e-3, integration_time=30e-3, 
                                             hanger = True, verbose=True, allocated_buffers=100,
                                             post_integration_delay=50e-6)
)

#note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_spec/{SAMPLE}', so, live_plot=True)
    
station.TWPA.off()

In [ ]:
# COARSE SCAN VS FLUX USING MAGNETIC FIELD
cur_field_x = get_real_field_x()
station = qcodes.Station.default
station.hetsrc.RF.power(-60)
station.qubsrc.power(-25)
het_freqs = np.arange(4.535e9, 4.555e9, 1e6)
qubit_frequencies = np.arange(5.5e9, 10.5e9, 15e6)
B_x = np.arange(12.8e-3, 11.6e-3, -0.002e-3)

station.qubsrc.output_rf('ON')

station.TWPA.off()
ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3)

so = sweep(set_magnet_x, B_x)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                             qubit_frequencies, station.qubsrc, 
                                             time_bin=0.5e-3, integration_time=30e-3, 
                                             hanger = True, verbose=True, allocated_buffers=100,
                                             post_integration_delay=50e-6)
)

#note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_spec/{SAMPLE}', so, live_plot=True)
    
station.TWPA.off()

In [ ]:
dac_cutter14()

In [ ]:
get_real_field_x()

In [ ]:
# COARSE SCAN VS FLUX USING MAGNETIC FIELD
cur_field_x = get_real_field_x()
station = qcodes.Station.default
station.hetsrc.RF.power(-60)
station.qubsrc.power(-16)
het_freqs = np.arange(4.539e9,4.555e9,0.5e6)
qubit_frequencies = np.arange(5.5e9, 7.5e9, 5e6)
B_x = np.arange(0.8e-3, 0.0e-3, -0.002e-3)

station.qubsrc.output_rf('ON')

station.TWPA.off()
ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3)

so = sweep(set_magnet_x, B_x)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                             qubit_frequencies, station.qubsrc, 
                                             time_bin=0.5e-3, integration_time=30e-3, 
                                             hanger = True, verbose=True, allocated_buffers=100,
                                             post_integration_delay=50e-6)
)

#note that the two integration times must match!!

result = do_experiment(f'optimizedqubit_spec/{SAMPLE}', so, live_plot=True)
    
station.TWPA.off()

In [ ]:
get_real_field_x

In [ ]:
dac_cutter14()

## Plots

In [ ]:
from qcodes import load_by_id
id_number = 1106
ds1 = load_by_id(id_number)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
#ds2 = load_by_id(382)
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
#mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))


m = np.mean(mags1)
for i in range(mags1.shape[0]):
    mags1[i,:] = mags1[i,:] - np.mean(mags1[i,:])+m
    
#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure()
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [-0.05, 0.85, 1, 11.5], vmax = 0.0075, vmin = 0.0018)
plt.colorbar()
plt.title('id: %d' %id_number)
plt.xlabel('B_x (mT)')
plt.ylabel('qubit drive frequency (GHz)')

In [ ]:
from qcodes import load_by_id
id_number = 1106
ds1 = load_by_id(id_number)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
#ds2 = load_by_id(382)
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
#mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))


m = np.mean(mags1)
for i in range(mags1.shape[0]):
    mags1[i,:] = mags1[i,:] - np.mean(mags1[i,:])+m
    
#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure()
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [-0.05, 0.85, 1, 11.5], vmax = 0.0075, vmin = 0.0018)
plt.colorbar()
plt.title('id: %d' %id_number)
plt.xlabel('B_x (mT)')
plt.ylabel('qubit drive frequency (GHz)')

In [ ]:
from qcodes import load_by_id
id_number = 975
ds1 = load_by_id(id_number)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('signal_magnitude')
field1 = np.squeeze(ds1.get_data('magnet_x'))
#ds2 = load_by_id(382)
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
#mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))


# m = np.mean(mags1)
# for i in range(mags1.shape[0]):
#     mags1[i,:] = mags1[i,:] - np.mean(mags1[i,:])+m
    
#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure()
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.imshow(np.transpose(mags1)[-1:0:-1],aspect='auto',extent = [field1[0], field1[-1],4.5, 4.6], vmax = 0.007, vmin = 0.0002)
cbar=plt.colorbar()
plt.title('id: %d' %id_number)
plt.xlabel('B_x (mT)')
plt.ylabel('frequency (GHz)')
cbar.set_label('S21 (V)')

In [ ]:
from qcodes import load_by_id
id_number = 792
ds1 = load_by_id(id_number)
freqs1 = ds1.get_data('frequency')
mags1 = ds1.get_data('magnitude')
cutter1 = np.squeeze(ds1.get_data('ivvi_dac2'))
#ds2 = load_by_id(382)
#freqs2 = ds2.get_data('frequency')
#mags2 = ds2.get_data('magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
#mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))


# m = np.mean(mags1)
# for i in range(mags1.shape[0]):
#     mags1[i,:] = mags1[i,:] - np.mean(mags1[i,:])+m
    
#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure()
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.imshow(np.transpose(mags1)[-1:0:-1],aspect='auto',extent = [cutter1[0], cutter1[-1], 4.52, 4.64], vmax = 0, vmin = -60)
cbar = plt.colorbar()
plt.title('id: %d' %id_number)
plt.xlabel('cutter (mV)')
plt.ylabel('frequency (GHz)')
cbar.set_label('S21 (dBm)')

In [ ]:
from qcodes import load_by_id
id_number = 971
ds1 = load_by_id(id_number)
field1 = np.squeeze(ds1.get_data('magnet_x'))
freqs1 = np.squeeze(ds1.get_data('frequency'))
mags1 = ds1.get_data('signal_magnitude')
ds2 = load_by_id(id_number+1)
field2 = np.squeeze(ds2.get_data('magnet_x'))
freqs2 = np.squeeze(ds2.get_data('frequency'))
mags2 = ds2.get_data('signal_magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))[::-1]

df1 = freqs1[0][1] - freqs1[0][0]
df2 = freqs2[0][1] - freqs2[0][0]

freqs = np.concatenate((freqs2, freqs1), axis=1)
mags = np.concatenate((mags2, mags1), axis=1)

    
field = np.zeros(np.shape(freqs))
for i in range(np.shape(freqs)[0]):
    field[i,:] = 1000*field1[i]
    

#mags = mags1
#freqs = 1e-9*freqs1
# #field = 1000*field1
# mags = mags2
# freqs = 1e-9*freqs2

    
    
m = np.mean(mags)
for i in range(mags.shape[0]):
    mags[i,:] = mags[i,:] - np.mean(mags[i,:]) + m
 
mags = mags[:-1, :-1]
    
#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure(figsize=(5,6))
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.pcolormesh(field, freqs, mags, vmin = 0.0005, vmax = 0.0040)
cbar =plt.colorbar()
cbar.set_label('S21 (V)')
plt.title('id: %d' %id_number)
plt.xlabel('B_x (mT)')
plt.ylabel('qubit drive frequency (GHz)')
filename = 'C:/Users/Administrator/Desktop/tests-marta/test1.png'
plt.savefig(filename, bbox_inches = 'tight', transparent=True)
plt.show()

In [ ]:
from qcodes import load_by_id
id_number = 971
shift = 3
ds1 = load_by_id(id_number)
# field1 = np.squeeze(ds1.get_data('magnet_x'))
freqs1 = 1e-9*np.squeeze(ds1.get_data('frequency'))
mags1 = ds1.get_data('signal_magnitude')
ds2 = load_by_id(id_number+1)
# field2 = np.squeeze(ds2.get_data('magnet_x'))
freqs2 = 1e-9*np.squeeze(ds2.get_data('frequency'))
mags2 = ds2.get_data('signal_magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))[::-1]

mags2 = mags2[shift-1:-1,:]
mags1 = mags1[0:-shift,:]
freqs2 = freqs2[shift-1:-1,:]
freqs1 = freqs1[0:-shift,:]

# df1 = freqs1[0][1] - freqs1[0][0]
# df2 = freqs2[0][1] - freqs2[0][0]

freqs = np.concatenate((freqs2, freqs1), axis=1)
mags = np.concatenate((mags2, mags1), axis=1)

    
field1 = np.linspace(-0.13, 1.43, np.shape(freqs)[0])
    
field = np.zeros(np.shape(freqs))
for i in range(np.shape(freqs)[0]):
    field[i,:] = field1[i]    

#mags = mags1
#freqs = 1e-9*freqs1
# #field = 1000*field1
# mags = mags2
# freqs = 1e-9*freqs2

    
    
m = np.mean(mags)
for i in range(mags.shape[0]):
    mags[i,:] = mags[i,:] - np.mean(mags[i,:]) + m
 
mags = mags[:-1, :-1]

#plt.figure()
#for i in range(1):
#    #plt.plot(freqs2[i][0][:],mags2[i][:], 'c')
#    plt.plot(freqs1[i][0][:],160+20*np.log(mags1[i][:]), 'b')
#    plt.grid()
#plt.xlim(3e9,8e9)


plt.figure(figsize=(7,6))
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.pcolormesh(field, freqs, mags, vmin = 0.0005, vmax = 0.0040)
cbar =plt.colorbar()
cbar.set_label('S21 (V)')
#plt.title('id: %d' %id_number)
plt.xlabel(r"flux ($\Phi_{0}$)")
plt.ylabel('qubit drive frequency (GHz)')
#plt.grid()
filename = 'C:/Users/Administrator/Desktop/tests-marta/test5.png'
plt.savefig(filename, bbox_inches = 'tight', transparent=True)
plt.show()

In [ ]:
from qcodes import load_by_id
id_number = 1024
shift = 3
ds1 = load_by_id(id_number)
# field1 = np.squeeze(ds1.get_data('magnet_x'))
freqs1 = 1e-9*np.squeeze(ds1.get_data('frequency'))
mags1 = ds1.get_data('signal_magnitude')
ds2 = load_by_id(id_number+2)
# field2 = np.squeeze(ds2.get_data('magnet_x'))
freqs2 = 1e-9*np.squeeze(ds2.get_data('frequency'))
mags2 = ds2.get_data('signal_magnitude')
mags1=np.reshape(mags1,(np.shape(mags1)[0],np.shape(mags1)[2]))
mags2=np.reshape(mags2,(np.shape(mags2)[0],np.shape(mags2)[2]))[::-1]

mags2 = mags2[shift-1:-1,:]
mags1 = mags1[0:-shift,::-1]
freqs2 = freqs2[shift-1:-1,:]
freqs1 = freqs1[0:-shift,::-1]

# df1 = freqs1[0][1] - freqs1[0][0]
# df2 = freqs2[0][1] - freqs2[0][0]

freqs = np.concatenate((freqs1, freqs2), axis=1)
mags = np.concatenate((mags1, mags2), axis=1)

    
field1 = np.linspace(-0.13, 1.415, np.shape(freqs)[0])
    
field = np.zeros(np.shape(freqs))
for i in range(np.shape(freqs)[0]):
    field[i,:] = field1[i]    

mags = mags[:-1, :-1]
  
    
m = np.mean(mags)
for i in range(mags.shape[0]):
    mags[i,:] = mags[i,:] - np.mean(mags[i,:]) + m



plt.figure(figsize=(7,6))
#plt.imshow(np.transpose(mags1)[-1:0:-1,-1:0:-1],aspect='auto',extent = [0.095, 0.6, 6, 10], vmax = 0.0045, vmin = 0.0015)
plt.pcolormesh(field, freqs, mags, vmin = 0.0005, vmax = 0.0025)
cbar =plt.colorbar()
cbar.set_label('S21 (V)')
#plt.title('id: %d' %id_number)
plt.xlabel(r"flux ($\Phi_{0}$)")
plt.ylabel('qubit drive frequency (GHz)')
#plt.grid()
filename = 'C:/Users/Administrator/Desktop/tests-marta/test6.png'
plt.savefig(filename, bbox_inches = 'tight', transparent=True)
plt.show()

In [ ]:
np.shape(mags1), np.shape(mags2), np.shape(mags), np.shape(freqs1), np.shape(freqs2), np.shape(freqs)

In [ ]:
plt.plot(freqs[0])

# Going up in field

In [ ]:
station.mgnt.GRPX.field_ramp_rate(1e-3)
station.mgnt.GRPY.field_ramp_rate(1e-3)
station.mgnt.GRPZ.field_ramp_rate(1e-3)

In [ ]:
set_magnet_x(0.0e-3)
set_magnet_y(0.0)
set_magnet_z(0.0e-3)

In [ ]:
get_real_field_x(), get_real_field_y(), get_real_field_z()

## Z 

### Going up in field using Oxford in QT2 DO NOT USE THIS!

In [ ]:
cur_field_z = get_real_field_z()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.53e9, 4.59e9, 15e6)
spec_src=station.hetsrc

station.qubsrc.output_rf('OFF')

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(set_magnet_z, np.arange(cur_field_z , 300.0e-3, -20e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_B_z/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
## Ramping B_z maintaining B_x relatively well aligned in-plane( 1mT down in B_x for every 50 mT down in B_z)

cur_field_z = get_real_field_z()
set_magnet_x(cur_field_z/50)
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.56e9, 4.6e9, 10e6)
spec_src=station.hetsrc

station.qubsrc.output_rf('OFF')

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(set_magnet_z_aligned, np.arange(cur_field_z , 0, -20e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_B_z_aligned/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

### Going up in field using AMIs

In [ ]:
station.ivvi.set_dacs_zero()

In [ ]:
station.RF.off()

In [ ]:
cur_field_z = station.z_mgnt.field()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(5.21e9, 5.24e9, 2e6)
spec_src=station.hetsrc

station.qubsrc.off()

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(station.z_mgnt.field, np.arange(cur_field_z , 1000.0e-3, 2e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_B_z/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
station.z_mgnt.field()

## X (we no longer use the AMI power supplies for X because it is too coarse!)

In [ ]:
cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(4.576e9, 4.582e9, 0.5e6)
spec_src=station.hetsrc

station.qubsrc.output_rf('OFF')


ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(set_magnet_x, np.arange(5e-3, 7e-3, +0.02e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_B_x/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

In [ ]:
# At a single frequency

cur_field_x = get_real_field_x()
station.hetsrc.RF.power(-55)
station.hetsrc.frequency(4.55e9)
station.qubsrc.output_rf('OFF')
B_x = np.arange(+23.4e-3, 25e-3, +0.05e-3)

ss.setup_single_averaged_IQpoint(0.1e-3, 30e-3, setup_awg=True)
so = sweep(set_magnet_x, B_x)(
    measure(ss.get_single_averaged_IQpoint_chanA),
)

#station.TWPA.on()
result = do_experiment(f"resonator_B_y/{SAMPLE}", so, live_plot=True)
#station.TWPA.off()

## Y

In [ ]:
cur_field_y = station.y_mgnt.field()
station.hetsrc.RF.power(-55)
spec_frequencies=np.arange(5.21e9, 5.30e9, 1e6)
spec_src=station.hetsrc

station.qubsrc.off()

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(station.y_mgnt.field, np.arange(cur_field_y , -0.1e-3, -0.1e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=False, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f"resonator_B_y/{SAMPLE}", so, live_plot=True)
station.TWPA.off()

### Two-tone vs B_Y

In [ ]:
dac_cutter14(-50)

In [ ]:
# Without finding resonator
cur_field_y = get_real_field_y()
station.hetsrc.RF.power(-55)
station.hetsrc.frequency(4.55e9)
qubit_frequencies = np.arange(6.6e9, 7.5e9, 5e6)
station.qubsrc.power(-25)
B_y = np.arange(cur_field_y, cur_field_y+1e-3, +0.1e-3)
spec_src=station.qubsrc

spec_src.output_rf('ON')

ctl = ss.setup_soft_sweep(qubit_frequencies, spec_src.frequency,
                          time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

station.TWPA.off()
so = sweep(set_magnet_y, B_y)(
    ss.measure_soft_time_avg_spec(qubit_frequencies, spec_src, time_bin=0.5e-3, integration_time=30e-3,
                                  setup_awg=False, verbose=True, allocated_buffers=100,
                                  post_integration_delay=100e-6)
)

result = do_experiment(f'qubit_spec_vs_B_y/{SAMPLE}', so, live_plot=True)
station.TWPA.off()

In [ ]:
# Finding resonator
station = qcodes.Station.default
station.hetsrc.RF.power(-55)
cur_field_y = get_real_field_y()
het_freqs = np.arange(4.543e9,4.550e9,1e6)
station.qubsrc.power(-25)
qubit_frequencies = np.arange(6.0e9, 7.5e9, 10e6)
B_y = np.arange(-0.7e-3, 3e-3, 0.1e-3)
spec_src=station.qubsrc


station.qubsrc.output_rf('ON')

station.TWPA.off()

ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, time_bin=0.5e-3, integration_time=30e-3, verbose=True, 
                          allocated_buffers=100, post_integration_delay=100e-6)

so = sweep(set_magnet_y, B_y)(
    ss.measure_qubit_spec_optimize_resonator(het_freqs, station.hetsrc, 
                                             qubit_frequencies, station.qubsrc, 
                                             time_bin=0.5e-3, integration_time=30e-3, 
                                             hanger=True, verbose=True, allocated_buffers=100,
                                             post_integration_delay=100e-6)
) 

#note that the two integration times must match!!

result = do_experiment(f'qubit_spec_vs_B_y/{SAMPLE}', so, live_plot=True)

station.TWPA.off()

# Time domain

In [ ]:
station.hetsrc.RF.power(-60)
station.hetsrc.frequency(4.5433e9)
station.qubsrc.power(-14)
station.qubsrc.frequency(6.014e9)

In [ ]:
import numpy as np
import qcodes as qc
import broadbean as bb
from broadbean.plotting import plotter

from pytopo.awg_sequencing import broadbean as bbtools
from pytopo.awg_sequencing.broadbean import BluePrints, BroadBeanSequence

ramp = bb.PulseAtoms.ramp
gaussian = bb.PulseAtoms.gaussian

## Amplitude Rabi

In [ ]:
class RabiSequence(BroadBeanSequence):
    """
    A sequence that consists of a single gaussian pulse followed by a readout pulse.

    required channels:
        'I' : analog output (qubit drive pulse)
        'ats_trigger' : marker for the alazar
        'ro_pulse' : readout marker 
        'qb_pulse' : qubit marker
    """
    name = 'rabi_sequence'

    def sequence(self, amplitudes, pulse_time, readout_time,cycle_time = 10e-6,
                 pre_pulse_time=1e-6, after_pulse_time=0.1e-6,
                 alazar_trigger_time=100e-9, marker_buffer=100e-9, cavity_lifetime = 0.2e-6):
        
        t_pulse = pre_pulse_time + pulse_time + after_pulse_time
        sigma = pulse_time/4.0
        #cycle_time=t_pulse+readout_time+1e-6
        
        
        elements = []
        for A in amplitudes:
            bps = bbtools.BluePrints(chan_map=self.chan_map, length=cycle_time, sample_rate=self.SR)
            
            bps['I'].insertSegment(0, ramp, (0, 0), dur=pre_pulse_time)
            bps['I'].insertSegment(1, gaussian, (A, sigma, 0, 0), 
                                   name='gaussian_pulse', dur=pulse_time)
            bps['I'].insertSegment(2, ramp, (0, 0), dur=cycle_time - pre_pulse_time - pulse_time)
            
            bps['qb_pulse'] = [(pre_pulse_time - marker_buffer, pulse_time + 2 * marker_buffer)]
            bps['ats_trigger'] = [(t_pulse+cavity_lifetime, alazar_trigger_time)]
            bps['ro_pulse'] = [(t_pulse, readout_time)]

            elements.append(bbtools.blueprints2element(bps))
        
        return bbtools.elements2sequence(elements, self.name)
    
    
RabiSequence.chan_map = {
    1 : ['I', 'ats_trigger', 'ro_pulse'],
    2 : [None, 'qb_pulse', None],   
}

RabiSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}




In [ ]:
station.qubsrc.on()
station.qubsrc.power(-20)

In [ ]:
# station.qubsrc.output_rf('OFF')
station.qubsrc.off()

In [ ]:
# station.qubsrc.on()

In [ ]:
readout_time = 3e-6
amplitudes = np.linspace(-0.5, 0.5, 21)
#amplitudes = np.array([0,0.5])

station.fg.ch1.state('OFF')
rabi_seq = RabiSequence(station.awg, SR=1e9)
rabi_seq.wait = 'all'
rabi_seq.setup_awg(amplitudes=amplitudes, pulse_time=1e-6, readout_time=readout_time, cycle_time = 10e-6,start_awg=True)

In [ ]:
station.fg.ch1.freq(1e4)

In [ ]:
station.fg.ch1.state('ON')
station.RF.power(-55)
station.hetsrc.frequency(4.0e9)
station.RF.on()

### Integrating the full readout

In [ ]:
station.fg.ch1.state('OFF')
station.qubsrc.off()
station.qubsrc.power(-20)
station.qubsrc.IQ_state('ON')
station.qubsrc.pulsemod_state('ON')
station.qubsrc.pulsemod_source('EXT')
station.awg.stop()
station.awg.start()

# post_iq_acq.integration_stop(readout_time+1e-6)
ctl = post_iq_acq
ctl.demod_frq(25e6)
# ctl = raw_acq
ctl.average_buffers(True)
ctl.setup_acquisition(None, amplitudes.size, 5000, acq_time=2e-6)
ctl.pre_acquire_func = lambda : station.fg.ch1.state('ON')
dataAvg = ctl.acquisition()


# tvals = ctl.tvals
tvals = ctl.demod_tvals
station.fg.ch1.state('OFF')

In [ ]:
fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes, np.abs(np.mean(np.squeeze(dataAvg), axis=0))[:,0], 'o')
ax.plot(np.abs(np.squeeze(dataAvg))[:,0], 'o')
# dataAvg.shape

#fig, ax = plt.subplots(1,1)
#ax.plot(amplitudes,np.mean(np.squeeze(dataAvg).imag, axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).real,axis=2), axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).imag,axis=2), axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:,].mean(axis=0).real)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:,].mean(axis=0).imag)
# ax.plot(tvals * 1e6, np.abs(np.squeeze(dataNoavg)[:,].mean(axis=0)))

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(amplitudes,np.abs(np.mean(np.squeeze(dataAvg), axis=0))[:,0],'o')
#ax.plot(np.abs(np.squeeze(dataAvg))[:,:,0])


#fig, ax = plt.subplots(1,1)
#ax.plot(amplitudes,np.mean(np.squeeze(dataAvg).imag, axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).real,axis=2), axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).imag,axis=2), axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:,].mean(axis=0).real)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:,].mean(axis=0).imag)
# ax.plot(tvals * 1e6, np.abs(np.squeeze(dataNoavg)[:,].mean(axis=0)))

###

In [ ]:
ctl = post_demod_acq
ctl.demod_frq(25e6)
# ctl = raw_acq
ctl.setup_acquisition(None, amplitudes.size, 1000, acq_time=20e-6)
ctl.pre_acquire_func = lambda : station.fg.ch1.state('ON')
dataNoavg = ctl.acquisition()


# tvals = ctl.tvals
tvals = ctl.demod_tvals
station.fg.ch1.state('OFF')

## Time Rabi

In [ ]:
class RabiSequence(BroadBeanSequence):
    """
    A sequence that consists of a single gaussian pulse followed by a readout pulse.

    required channels:
        'I' : analog output (qubit drive pulse)
        'ats_trigger' : marker for the alazar
        'ro_pulse' : readout marker 
        'qb_pulse' : qubit marker
    """
    name = 'rabi_sequence'

    def sequence(self, pulse_times, readout_time,
                 pre_pulse_time=1e-6, after_pulse_time=0.1e-6,
                 alazar_trigger_time=100e-9, marker_buffer=50e-9):
        
        
        elements = []
        for pulse_time in pulse_times:
            t_pulse = pre_pulse_time + pulse_time + after_pulse_time
            #sigma = pulse_time/4.0
            cycle_time=t_pulse+readout_time+1e-6
            
            bps = bbtools.BluePrints(chan_map=self.chan_map, length=cycle_time, sample_rate=self.SR)
            
            bps['I'].insertSegment(0, ramp, (0, 0), dur=pre_pulse_time)
            # GAUSSIAN PULSE
            #bps['I'].insertSegment(1, gaussian, (A, sigma, 0, 0), 
            #                      name='gaussian_pulse', dur=pulse_time)
            # SQUARE PULSE
            bps['I'].insertSegment(1, ramp, (0.5, 0.5), dur=pulse_time)
            
            bps['I'].insertSegment(2, ramp, (0, 0), dur=cycle_time - pre_pulse_time - pulse_time)
            
            
            bps['qb_pulse'] = [(pre_pulse_time - marker_buffer, pulse_time + 2 * marker_buffer)]
            bps['ats_trigger'] = [(t_pulse - after_pulse_time, alazar_trigger_time)]
            bps['ro_pulse'] = [(t_pulse, readout_time)]

            elements.append(bbtools.blueprints2element(bps))
        
        return bbtools.elements2sequence(elements, self.name)
    
    
RabiSequence.chan_map = {
    1 : ['I', 'ats_trigger', 'ro_pulse'],
    2 : [None, 'qb_pulse', None],   
}

RabiSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

In [ ]:
readout_time=10e-6
#pulse_times = np.logspace(-7, -4, 51)
pulse_times = np.linspace(1e-8, 1e-7, 51)
#amplitudes = np.array([0,0.5])

station.fg.ch1.state('OFF')
rabi_seq = RabiSequence(station.awg, SR=1e9)
rabi_seq.wait = 'all'
rabi_seq.setup_awg(pulse_times=pulse_times,  after_pulse_time=0.5e-6, readout_time=readout_time, start_awg=True)

In [ ]:
station.qubsrc.output_rf('ON')
station.qubsrc.power(20)

In [ ]:
station.qubsrc.output_rf('OFF')

In [ ]:
station.qubsrc.output_rf()

In [ ]:
post_iq_acq.integration_stop(readout_time+1e-6)
ctl = post_iq_acq
ctl.demod_frq(8e6)
# ctl = raw_acq
ctl.setup_acquisition(None, pulse_times.size,2, acq_time=0.8e-6)
ctl.pre_acquire_func = lambda : station.fg.ch1.state('ON')
dataAvg = ctl.acquisition()


# tvals = ctl.tvals
tvals = ctl.demod_tvals
station.fg.ch1.state('OFF')

In [ ]:
dataAvg.shape

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(pulse_times,np.abs(np.mean(np.squeeze(dataAvg), axis=0))[:,0],'o-')
#ax.plot(np.abs(np.squeeze(dataAvg))[:,:,0])


#fig, ax = plt.subplots(1,1)
#ax.plot(amplitudes,np.mean(np.squeeze(dataAvg).imag, axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).real,axis=2), axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).imag,axis=2), axis=0)[:,0])

# fig, ax = plt.subplots(1,1)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:,].mean(axis=0).real)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:,].mean(axis=0).imag)
# ax.plot(tvals * 1e6, np.abs(np.squeeze(dataNoavg)[:,].mean(axis=0)))

# TESTS Magnetic Field Sweeps

## TESTING UNDER DEVELOPMENT DO NOT USE UNLESS YOU ARE SURE

In [ ]:
station.mgnt.theta_target(0.0)
station.mgnt.phi_target(0)

In [ ]:
print('Measured field = ' + station.mgnt.field_measured().repr_spherical())
#print('Measured field = ' + station.mgnt.field_measured().repr_cartesian())
print('Target field = ' + station.mgnt.field_target().repr_spherical())
#print('Target field = ' + station.mgnt.field_target().repr_cartesian())

### Get resonator trace and plot

In [ ]:



#ctl = ss.setup_soft_sweep(het_freqs, station.hetsrc.frequency, integration_time=30e-3)
#testout = get_resonator_spec_and_fit(het_freqs)
#testout.params['center'].value, testout.params['center'].stderr


## Manual Magnet Sweeps

In [ ]:
station.mgnt.r_measured()

In [ ]:
#station.mgnt.field_ramp_rate(FieldVector(x=1e-3, y=1e-3, z=1e-3)) ##sweep r


cur_r = station.mgnt.r_measured()
station.hetsrc.RF.power(-30)
spec_frequencies=np.arange(5.293e9, 5.315e9, 0.5e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.on()
so = sweep(set_magnet_r, np.arange(cur_r,0., -20e-3))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src.frequency, setup=False)
)

result = do_experiment(f"resonator_magnet_r/{SAMPLE}", so, live_plot=True)
station.TWPA.off()



In [ ]:
station.mgnt.theta_measured()

In [ ]:
cur_theta = station.mgnt.theta_measured() ##sweep theta
station.hetsrc.RF.power(-70)
spec_frequencies=np.arange(5.290e9, 5.315e9, 0.5e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src.frequency, integration_time=30e-3)

station.TWPA.on()
so = sweep(set_magnet_theta, np.arange(cur_theta,cur_theta+0.1, 0.005))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src.frequency, setup=False)
)

result = do_experiment(f"resonator_magnet_theta/{SAMPLE}", so, live_plot=True)
station.TWPA.off()



In [ ]:
station.mgnt.r_measured() > 0.05

In [ ]:
if station.mgnt.r_measured() > 0.05:
    cur_phi = station.mgnt.phi_measured() ##makes sure you don't get jumps
else:
    cur_phi = station.mgnt.phi_target() #makes sure phi doesn't change sign when it is not well defined
station.hetsrc.RF.power(-70)
spec_frequencies=np.arange(5.295e9, 5.315e9, 0.5e6)
spec_src=station.hetsrc

ctl = ss.setup_soft_sweep(spec_frequencies, spec_src, integration_time=30e-3)

station.TWPA.on()
so = sweep(set_magnet_phi, np.arange(cur_phi,cur_phi-4, -0.6))(
    ss.measure_soft_time_avg_spec(spec_frequencies, spec_src, setup=False)
)

result = do_experiment(f"resonator_magnet_phi/{SAMPLE}", so, live_plot=True)
station.TWPA.off()



## Automated Magnet Sweeps

In [ ]:
from pytopo.field_optimization.movement import FieldOptimizationProblem ##optimize at fixed mag

In [ ]:
from pytopo.field_optimization.movement import FieldOptimizationProblem ##optimize at fixed mag


xvals = np.arange(5.29e9,5.32e9,0.5e6)
    
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('X values')
ax.set_ylabel('Y values')
line = ax.plot(xvals, np.zeros_like(xvals), '.-')[0]

ctl = ss.setup_soft_sweep(xvals, station.hetsrc.frequency, integration_time=30e-3)

# now we wrap the measure function such that it doesn't need arguments anymore
# of course this requires that we have already good constant values for ax and line
measure_trace = partial(measure_and_plot_resonator, ax, line)

# now we just have to call the new function wrapper without arguments


field_opt = FieldOptimizationProblem(station.mgnt, measure_trace, get_resonator_FWHM)



cur_r = station.mgnt.r_target()
cur_theta = station.mgnt.theta_target()
cur_phi = station.mgnt.phi_target()

with station.TWPA():
    opt, extra = field_opt.optimize_at_fixed_magnitude(
        cur_r, (cur_phi-3, cur_phi+3), 2,(cur_theta-0.1, cur_theta+0.1), 2,
        plot=True, return_extra=True, verbose=True, ramp_rate=1e-3, n_steps=3
    )
# station.TWPA.off()

In [ ]:
0.4/50/2*1e3

In [ ]:
station.RF.power(-30)
xvals = np.arange(5.285e9,5.317e9,0.4e6)
    
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel('X values')
ax.set_ylabel('Y values')
line = ax.plot(xvals, np.zeros_like(xvals), '.-')[0]

ctl = ss.setup_soft_sweep(xvals, station.hetsrc.frequency, integration_time=30e-3)

# now we wrap the measure function such that it doesn't need arguments anymore
# of course this requires that we have already good constant values for ax and line
measure_trace = partial(measure_and_plot_resonator, ax, line)

# now we just have to call the new function wrapper without arguments


field_opt = FieldOptimizationProblem(station.mgnt, measure_trace, get_resonator_FWHM)



cur_r = station.mgnt.r_measured()
cur_theta = station.mgnt.theta_measured()
cur_phi = station.mgnt.phi_measured()
0.005

#with station.TWPA():
optimum, extra = field_opt.optimize_and_ramp_magnitude(
    cur_r, 1e-3, 200,
    cur_phi, 0, 1,
    cur_theta, 0.01, 5,
    reoptimization_threshold=4.5,
    ramp_rate=3e-3,
    n_steps=5,
    return_extra=True
    )

# station.TWPA.off()

In [ ]:
plt.figure()
plt.plot(
    [field.phi for field in extra['history']['optima']],
    [field.theta for field in extra['history']['optima']],
    zorder=-10
)
plt.scatter(
    [field.phi for field in extra['history']['optima']],
    [field.theta for field in extra['history']['optima']],
    c=[field.r for field in extra['history']['optima']]
)
plt.colorbar()

In [ ]:
plt.figure()
plt.plot(extra['history']['objectives'])

# Test Code for Writing Time Domain Sequences

In [ ]:
import numpy as np
import qcodes as qc
import broadbean as bb
from broadbean.plotting import plotter

from pytopo.awg_sequencing import broadbean as bbtools
from pytopo.awg_sequencing.broadbean import BluePrints, BroadBeanSequence

ramp = bb.PulseAtoms.ramp
sine = bb.PulseAtoms.sine
gaussian = bb.PulseAtoms.gaussian

In [ ]:
class RabiSequence(BroadBeanSequence):
    """
    A sequence that consists of a single gaussian pulse followed by a readout pulse.

    required channels:
        'I' : analog output (qubit drive pulse)
        'ats_trigger' : marker for the alazar
        'ro_pulse' : readout marker 
        'qb_pulse' : qubit marker
    """
    name = 'rabi_sequence'

    def sequence(self, amplitudes, pulse_time, readout_time,
                 pre_pulse_time=1e-6, after_pulse_time=0.1e-6,
                 alazar_trigger_time=100e-9, marker_buffer=100e-9):
        
        t_pulse = pre_pulse_time + pulse_time + after_pulse_time
        sigma = pulse_time/4.0
        cycle_time=t_pulse+readout_time+1e-6
        
        elements = []
        for A in amplitudes:
            bps = bbtools.BluePrints(chan_map=self.chan_map, length=cycle_time, sample_rate=self.SR)
            
            bps['I'].insertSegment(0, ramp, (0, 0), dur=pre_pulse_time)
            bps['I'].insertSegment(1, gaussian, (A, sigma, 0, 0), 
                                   name='gaussian_pulse', dur=pulse_time)
            bps['I'].insertSegment(2, ramp, (0, 0), dur=cycle_time - pre_pulse_time - pulse_time)
            
            bps['qb_pulse'] = [(pre_pulse_time - marker_buffer, pulse_time + 2 * marker_buffer)]
            bps['ats_trigger'] = [(t_pulse - after_pulse_time, alazar_trigger_time)]
            bps['ro_pulse'] = [(t_pulse, readout_time)]

            elements.append(bbtools.blueprints2element(bps))
        
        return bbtools.elements2sequence(elements, self.name)
    
    
RabiSequence.chan_map = {
    1 : ['I', 'ats_trigger', 'ro_pulse'],
    2 : [None, 'qb_pulse', None],   
}

RabiSequence.chan_settings = {
    1 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
    2 : {
        'marker_hi' : [1.5, 1.5],
        'marker_lo' : [-0.3, -0.3],
        },
}

fg.ch1.state('OFF')
readout_time=5e-6
amplitudes = np.linspace(-0.5, 0.5, 10)
rabi_seq = RabiSequence(station.awg, SR=1e9)
rabi_seq.wait = 'all'
rabi_seq.setup_awg(amplitudes=amplitudes, pulse_time=1e-6, readout_time=readout_time, start_awg=True)

In [ ]:
post_iq_acq.integration_stop(readout_time+1e-6)

In [ ]:
station.hetsrc.RF.power(-50)
station.LO.power(10.0)
station.hetsrc.IF(8e6)
station.hetsrc.frequency(4.53e9)

In [ ]:
ctl = post_iq_acq
ctl.demod_frq(8e6)
# ctl = raw_acq
ctl.setup_acquisition(None, amplitudes.size, 1000, acq_time=6e-6)
ctl.pre_acquire_func = lambda : fg.ch1.state('ON')
dataAvg = ctl.acquisition()


# tvals = ctl.tvals
tvals = ctl.demod_tvals

In [ ]:
ctl = post_demod_acq
ctl.demod_frq(8e6)
# ctl = raw_acq
ctl.setup_acquisition(None, amplitudes.size, 1000, acq_time=6e-6)
ctl.pre_acquire_func = lambda : fg.ch1.state('ON')
dataNoavg = ctl.acquisition()


# tvals = ctl.tvals
tvals = ctl.demod_tvals

In [ ]:
data.shape

In [ ]:
dataNoavg.shape

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(amplitudes,np.mean(np.squeeze(dataAvg).real, axis=0)[:,0])

fig, ax = plt.subplots(1,1)
ax.plot(amplitudes,np.mean(np.squeeze(dataAvg).imag, axis=0)[:,0])

fig, ax = plt.subplots(1,1)
ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).real,axis=2), axis=0)[:,0])

fig, ax = plt.subplots(1,1)
ax.plot(amplitudes,np.mean(np.mean(np.squeeze(dataNoavg).imag,axis=2), axis=0)[:,0])

fig, ax = plt.subplots(1,1)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:, :, 0].mean(axis=0).real)
# ax.plot(tvals * 1e6, np.squeeze(dataNoavg)[:, :, 0].mean(axis=0).imag)
# ax.plot(tvals * 1e6, np.abs(np.squeeze(dataNoAvg)[:, :, 0].mean(axis=0)))

# OLD STUFF BELOW - ONLY REFERENCE

In [ ]:
frqs, data = soft_resonator_spec(5.6e9, 5.7e9, 1e6, RF_power=-70)

## Resonator vs time

In [ ]:
start, stop, step = 5.6e9, 5.7e9, 2e6
integration_time = 10e-3
trigger_interval = 1e-3

exp = new_experiment(name='resonator_spec_vs_time', sample_name=qc.config.user.experiment.sample_name)

# set basic RF stuff
station.hetsrc.RF.power(-70)

ctl = station.post_iq_acq
ctl.average_buffers(True)
ctl.demod_frq(station.hetsrc.IF())

SR = station.alazar.sample_rate()
t_buf = trigger_interval - 10e-6
nbufs = int(integration_time // trigger_interval)

acq_size = 2 * integration_time * SR * 1e-6 # in MB
buf_size = 2 * t_buf * SR * 1e-6
print(f'acquisition size: {acq_size} MB; {buf_size} MB/buffer.')  
print(f'acquisition time/buffer: {t_buf*1e6} us; {nbufs} buffers.')

ctl.setup_acquisition(
    samples=int(t_buf * SR // 64 * 64),
    records=1,
    buffers=nbufs,
    allocated_buffers=10,
)

tvals = np.arange(100)
frqs = np.arange(start, stop+step, step)
data = np.zeros((frqs.size, 2), dtype=np.complex64)

m = acquisition_tools.AlazarMeasurement(
    station=station, exp=exp, 
    soft_params=['iteration', station.hetsrc.frequency],
    hard_axes=[
        ('channel', np.array([1,2])),
    ],
    imag=False, real=False, mag=True, phase=True,
)

with m.run() as datasaver:

    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )

    for j, it in enumerate(tvals):
        for i, f in enumerate(frqs):
            print(f'Current iteration: {it}. Current frequency: {f*1e-6:.2f} MHz', end='\r')
            station.hetsrc.frequency(f)
            time.sleep(0.01)

            _data = ctl.acquisition()[0,0,0,:]
            data[i,:] = _data
            mag, phase = np.abs(_data), np.angle(_data, deg=True)

            result = m.get_result(_data, (station.hetsrc.frequency, f), ('iteration', it))
            datasaver.add_result(*result)

# Gate response

In [ ]:
station.hetsrc.frequency(5.647e9)
station.hetsrc.RF.power(-70)
SR = station.alazar.sample_rate()
dac = station.ivvi.dac1
dacl = station.ivvi.dac11
dacr = station.ivvi.dac13

station = qc.Station.default
exp = new_experiment(name='gate_response', sample_name=qc.config.user.experiment.sample_name)

ctl = station.post_iq_acq
ctl.average_buffers(True)
ctl.demod_frq(station.hetsrc.IF())

ctl.setup_acquisition(
        samples=int(980e-6 * SR // 64 * 64),
        records=1,
        buffers=10,
        allocated_buffers=10,
    )

gatevals = np.arange(0, 1000, 0.5)

m = acquisition_tools.AlazarMeasurement(
        station=station, exp=exp, 
        soft_params=[dacl],
        hard_axes=[
            ('channel', np.array([1,2])),
        ],
        imag=False, real=False, mag=True, phase=True,
    )

with m.run() as datasaver:
        
    datasaver.dataset.subscribe(
        QcodesDatasetSubscriber(datasaver.dataset), 
        state=[], min_wait=0, min_count=1,
    )

    for i, g in enumerate(gatevals):
        print(f'Current gate voltage: {g} mV', end='\r')
        dacl(int(g))
        dacr(int(g))
        dac(int(g))

        _data = ctl.acquisition()[0,0,0,:]
        mag, phase = np.abs(_data), np.angle(_data, deg=True)

        result = m.get_result(_data, (dacl, g))
        datasaver.add_result(*result)

# Qubit spec

# Scribble

In [ ]:
f"{123456:1.3e}"

In [ ]:
raw_acq = instools.create_inst(acquisition_controllers.RawAcqCtl, 
                                   'raw_acq', 'alazar',
                                   allocate_samples=5e8, 
                                   force_new_instance=True)

In [ ]:
with station.alazar.syncing():
    station.alazar.samples_per_record(int(100e-3/10e-9)//128 * 128)
    station.alazar.records_per_buffer(1)
    station.alazar.buffers_per_acquisition(2)
    
raw_acq.setup_acquisition(samples=int(100e-3/10e-9)//128 * 128,
                          records=1, buffers=2)
data = raw_acq.acquisition()

In [ ]:
data2 = np.squeeze(data[:,1,...,0])
tvals = raw_acq.tvals

In [ ]:
fig, ax = plt.subplots(1,1)
ax.plot(tvals, data2)

In [ ]:
fft = np.fft.rfft(data2)
frqs = np.fft.rfftfreq(data2.size, d=tvals[1]-tvals[0])

fig, ax = plt.subplots(1,1)
ax.plot(frqs, np.abs(fft))